# Combat stats
This is to determine the average statistics for combat within the game. On average monsters should deal about 1/3 of the player's health in damage. They should instakill a player no more than 2% of the time.

Monsters average stats are calculated using those advertised under Making Monsters. Players stats are calculated by a combination of what average stats would be, and expected equipment purchases (especially for armor).

## Set Averages
The average stats for monsters and players by level.

In [1]:
def dice(rolls, sides):
    return {
        'rolls': rolls,
        'sides': sides
    }

In [2]:
damage_dice_by_level = {
    1: [dice(1, 8), dice(1, 6)],
    2: [dice(2, 8)],
    3: [dice(1, 10), dice(1, 8)],
    4: [dice(2, 12), dice(1, 4)],
    5: [dice(2, 12), dice(1, 8)],
    6: [dice(1, 20), dice(2, 8)],
    7: [dice(2, 20)],
    8: [dice(2, 20), dice(1, 8)],
    9: [dice(2, 20), dice(1, 12)],
    10:[dice(3, 20)]
    
}

player_damage_dice = [dice(1, 10), dice(1, 4)]

attack_bonus_by_level = {
    1: 4,
    2: 4,
    3: 5,
    4: 7,
    5: 7,
    6: 9,
    7: 12,
    8: 12,
    9: 13,
    10: 14
}

In [29]:
monster_armor_by_level = {
    1: 8,
    2: 9,
    3: 11,
    4: 13,
    5: 16,
    6: 18,
    7: 21,
    8: 23,
    9: 26,
    10: 28
}

monster_health_by_level = {
    1: 21,
    2: 26,
    3: 30,
    4: 34,
    5: 38,
    6: 42,
    7: 45,
    8: 48,
    9: 50,
    10: 55
}

In [25]:
def check_pos_int(values):
    """Check if a value is a positive integer
    
    Args:
        values: (List) values to be checked for validity
    """
    for value in values:
        assert value >= 0, 'Value must be a nonnegative integer'

check_pos_int([1, 2, 3])

In [4]:
def player_health(level):
    check_pos_int([level])
    level_ups = level - 1
    health = 15
    health += level_ups * 3
    health += 5 * int(level_ups / 3)
    return health

assert(player_health(1) == 15)
assert(player_health(2) == 18)
assert(player_health(4) == 29)
assert(player_health(9) == 49)
assert(player_health(10) == 57)

In [5]:
def player_armor(level):
    check_pos_int([level])
    if level <= 3:
        return 9
    if level <= 5:
        return 14
    if level <= 6:
        return 15
    if level <= 7:
        return 16
    return 19
    
assert(player_armor(1) == 9)
assert(player_armor(3) == 9)
assert(player_armor(6) == 15)
assert(player_armor(9) == 19)

## Attack
Functions needed to calculate the power of a monster attack.

In [6]:
import random

def get_attack(damage_dice, bonus):
    """ calculate the power of an attack
    
    Args:
        damage_dice (list of damage_die): The dice to be rolled
        bonus (int): The static bonus added to attacks     
    
    Return: (int) power of an attack
    """
    check_pos_int([bonus])
    total = 0
    largest_maxed = 0
    for damage_die in damage_dice:
        sides = damage_die['sides']
        for i in range(damage_die['rolls']):
            roll = random.randint(1, sides)
            total += roll
            if roll == sides:
                largest_maxed = max(largest_maxed, sides)
    if largest_maxed > 0:
       total += random.randint(1, largest_maxed) 
    return total + bonus

for i in range (1000):
    assert(get_attack([dice(1, 4)], 0) in range(1,9))
    assert(get_attack([dice(2, 6)], 0) in range(2,25))
    assert(get_attack([dice(1, 8)], 0) in range(1,17))
    assert(get_attack([dice(2, 12)], 0) in range(2,49))

In [7]:
def damage_percentage(attack, level):
    """Calculate the percentage of health lost
    
    Args:
        All are positive ints
        
    Returns:
        Health remaining after the attack
    """
    check_pos_int([attack, level])
    armor = player_armor(level)
    health = player_health(level)
    if attack <= armor:
        return 0
    damage = attack - armor
    if damage >= health:
        return 100
    return (damage / health) * 100

assert(damage_percentage(12, 1) == 20)
assert(damage_percentage(18, 1) == 60)
assert(damage_percentage(38, 5) == 75)
assert(damage_percentage(42, 8) == 50)

In [8]:
def combat_stats(level):
    '''
    Calculate the stats of combat.
    
    Args:
        Level (int)
    
    Return: (list of float) [average damage dealt, hits that one shotted player, misses]
    '''
    total_damage_percent = 0
    instakills = 0
    missed = 0
    iterations = 100000
    for i in range(iterations):
        attack = get_attack(damage_dice_by_level[level],
                       attack_bonus_by_level[level])
        damage_percent = damage_percentage(attack, level)
        total_damage_percent += damage_percent
        if damage_percent == 100:
            instakills += 1
        elif damage_percent == 0:
            missed += 1
    stats = [total_damage_percent, instakills, missed]
    return [stat / iterations for stat in stats]

In [9]:
for level in range(1, 11):
    print(combat_stats(level))

[28.541533333333025, 0.01583, 0.20788]
[29.029555555548864, 0.00864, 0.15591]
[33.71114285714642, 0.01396, 0.07479]
[34.729448275865884, 0.00744, 0.05457]
[37.6198125, 0.00638, 0.03052]
[42.7344571428573, 0.02383, 0.01594]
[41.56674418604275, 0.02422, 0.01494]
[43.46950000000254, 0.02027, 0.01068]
[46.701265306124846, 0.02257, 0.00412]
[48.9167894736816, 0.02122, 0.00135]


## Player Damage

In [40]:
def player_attack(damage_dice, level, advantage=False):
    total = 0
    iterations = 10000
    for i in range(iterations):
        total += get_attack(damage_dice, (level * 2.5) + 6)
    return total / iterations


def player_damage(damage_dice, level, advantage=False):
    total = 0
    iterations = 10000
    monster_armor = monster_armor_by_level[level]
    for i in range(iterations):
        attack = get_attack(damage_dice, (level * 2.5) + 6)
        if advantage:
            attack = max(attack, get_attack(damage_dice, bonus))
        total += max(0, attack - monster_armor)
    return total / iterations

def rounds_to_victory(level):
    damage_dice = player_damage_dice
    iterations = 10000
    total_rounds = 0
    attack_bonus = 6 + (level * 2.5)
    monster_armor = monster_armor_by_level[level]
    monster_health = monster_health_by_level[level] * 4
    for i in range(iterations):
        current_monster_health = monster_health
        rounds = 0
        while current_monster_health > 0:
            attack = get_attack(damage_dice, attack_bonus)
            damage = max(0, attack - monster_armor)
            current_monster_health -= damage
            rounds += .25
        total_rounds += rounds
    return total_rounds / iterations

In [41]:
for level in range(1, 10):
    rounds = rounds_to_victory(level)
    damage = 6 + (level * 2.5)
    attack = player_attack(player_damage_dice, level)
    print(f'Players on average have attack: {attack} deal {damage} and win in {rounds}')

Players on average have attack: 17.6188 deal 8.5 and win in 2.329175
Players on average have attack: 20.1242 deal 11.0 and win in 2.47475
Players on average have attack: 22.6012 deal 13.5 and win in 2.72025
Players on average have attack: 25.1135 deal 16.0 and win in 2.934525
Players on average have attack: 27.6136 deal 18.5 and win in 3.415425
Players on average have attack: 30.1032 deal 21.0 and win in 3.5937
Players on average have attack: 32.6453 deal 23.5 and win in 4.017575
Players on average have attack: 35.0405 deal 26.0 and win in 4.0928
Players on average have attack: 37.651 deal 28.5 and win in 4.4375


In [44]:
def average_roll(dice):
    iterations = 10000
    total = 0
    for i in range(iterations):
        total += get_attack(dice, 0)
    return total / iterations

In [48]:
print(average_roll([dice(1, 12), dice(1, 8)]))
print(average_roll([dice(1, 12)]))
print(average_roll([dice(1, 10), dice(1, 4)]))
print(average_roll([dice(1, 8), dice(1, 4)]))
print(average_roll([dice(1, 10)]))
print(average_roll([dice(1, 10)]))
print(average_roll([dice(2, 10)]))

12.0736
7.0638
9.1061
8.0891
6.0782
5.9838
11.9675
